# Scraping ESG Ratings for Flughafen Zürich AG

In [ ]:
import requests
from bs4 import BeautifulSoup

In [15]:
##fetching the url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.29'}
r = requests.get("https://finance.yahoo.com/quote/FHZN.SW/sustainability/", headers=headers)
print(r.status_code)

200


In [ ]:
print(r.content)

In [16]:
##searching for the elements containing the relevant information
soup = BeautifulSoup(r.content, "html.parser")

##1.Total ESG Score
for i in soup.find_all("div", {"class":"Fz(36px) Fw(600) D(ib) Mend(5px)"}):
    ESG_score = i.text

##2.percentile
for i in soup.find_all("span", {"class":"Bdstarts(s) Bdstartw(0.5px) Pstart(10px) Bdc($seperatorColor) Fz(12px) smartphone_Bd(n) Fw(500)"}):
    percentile = i.text

##3.level
for i in soup.find_all("div", {"class":"Fz(s) Fw(500) smartphone_Pstart(4px)"}):
    level = i.text

##4.environment risk score,##social risk score,##Governance risk score
rating = []
for i in soup.find_all("div", {"class":"D(ib) Mt(10px) smartphone_Mt(5px)"}):
    rating.append(i.text)

    
##creating a dict to store the information
my_dict = {'ESG_score': ESG_score, 'percentile':percentile, 'level':level, 
               'Environment_Risk_Score':rating[0],
               'Social_Risk_Score':rating[1],
               'Governance_Risk_Score':rating[2]}


In [17]:
##transforming the dict to a dataframe
import pandas as pd
df = pd.DataFrame.from_dict(my_dict, orient = 'index', columns = ['Flughafen Zürich AG'])

In [18]:
df

,Flughafen Zürich AG
ESG_score,23
percentile,34th percentile
level,Medium
Environment_Risk_Score,4.3
Social_Risk_Score,12.3
Governance_Risk_Score,6.6


# Adding ESG Ratings of other airport companies to the dataframe 

In [ ]:
##transpose the dataframe to add more rows
df_transposed = df.T

In [21]:
df = df_transposed

In [22]:
df.reset_index(inplace=True)
df.head()

,index,ESG_score,percentile,level,Environment_Risk_Score,Social_Risk_Score,Governance_Risk_Score
0,Flughafen Zürich AG,23,34th percentile,Medium,4.3,12.3,6.6


In [23]:
df.rename(columns={'index': 'Airport'}, inplace=True)
df

,Airport,ESG_score,percentile,level,Environment_Risk_Score,Social_Risk_Score,Governance_Risk_Score
0,Flughafen Zürich AG,23,34th percentile,Medium,4.3,12.3,6.6


In [198]:
##adding empty rows to fill up later
df.loc[df.shape[0]] = [None, None, None, None, None, None, None]

In [205]:
##filling up the rows
df.iloc[1] = ['Aéroports de Paris SA', 16.7 , None, 'Low', None, None, None]
df.iloc[2] = ['Airports of Thailand PCL', 16.6 , None, 'Low', None, None, None]
df.iloc[3] = ['Malaysia Airports Holdings Berhad', 20.1 , None, 'Medium', None, None, None]
df.iloc[4] = ['Flughafen Wien AG', 20 , None, 'Medium', None, None, None]
df.iloc[5] = ['Airport City Ltd', None , None, None , None, None, None]
df.iloc[6] = ['AIRPORT FACILITIES Co. LTD', None, None, None, None, None, None]

In [206]:
df

,Airport,ESG_score,percentile,level,Environment_Risk_Score,Social_Risk_Score,Governance_Risk_Score
0,Flughafen Zürich AG,23,34th percentile,Medium,4.3,12.3,6.6
1,Aéroports de Paris SA,16.7,None,Low,None,None,None
2,Airports of Thailand PCL,16.6,None,Low,None,None,None
3,Malaysia Airports Holdings Berhad,20.1,None,Medium,None,None,None
4,Flughafen Wien AG,20,None,Medium,None,None,None
5,Airport City Ltd,None,None,None,None,None,None
6,AIRPORT FACILITIES Co. LTD,None,None,None,None,None,None


In [207]:
df.to_csv('ESG_ratings_stage.csv')